In [ ]:
!pip install sentence_transformers
!pip install konlpy

     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 3.8 MB 11.7 MB/s 
     |████████████████████████████████| 1.2 MB 37.4 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 43.1 MB/s 
     |████████████████████████████████| 6.5 MB 32.7 MB/s 
     |████████████████████████████████| 895 kB 28.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=3435273e2d8d9da98029b3260a0a8e29ce4ee15793c9d8a6cf1c9fc1182d858c
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 36.6 MB/s 


#라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

#드라이브 마운트 & 파일 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
comu=pd.read_csv("/content/drive/Shareddrives/고대신문 프로젝트/기성언론 팀/데이터 분석 시각화/장수혁/(최종) 고려대학교 온라인 커뮤니티.csv")
news=pd.read_csv("/content/drive/Shareddrives/고대신문 프로젝트/기성언론 팀/데이터 분석 시각화/장수혁/(최종) 뉴스 신문기사.csv")

In [ ]:
del comu["Unnamed: 0"]

In [ ]:
comu.head()

,keyword,title,content,likes,date,comment,comment_likes,comment_num,content_preprocess,comment_preprocess,content_tokenize,comment_tokenize
0,조국,"""조국 땐 촛불, 교수 비리엔 입닫는 고대생""..총학 '성명 발표'",물어뜯기 바쁘네요 진짜 귀막고 얘기하는거 같음,55,2020-09-28,"*대깨문들의 이중논리 1. 박근혜 최순실에는 분노하면서, 추미애, 조국, 윤미향, ...",106,1,물어뜯기 바쁘네요 진짜 귀 막고 얘기하는 거 같음,대깨문들의 이 중 논리 1 박근혜 최순실에는 분 노하면서 추미애 조국 윤 미향 안민...,"['물어', '뜯기', '바쁘네요', '진짜', '귀', '막고', '얘기', '하...","['대', '깨문', '들', '의', '이', '중', '논리', '1', '박근..."
1,조국,"[속보]법원 ""조국 딸, 단국대 1저자 논문 기여 안했다""",이번에는 고려대가 반응할까요?,116,2020-12-23,이러고는 형이 확정된건 아니니 최종심까지는 지켜봐야한다 하면 졸업햇으니 자퇴는 안되...,111,1,이번에는 고려대가 반응할까요,이러고는 형이 확정된 건 아니니 최종심까지는 지켜봐야 한다 하면 졸업 햇으니 자퇴는...,"['이번', '에는', '고려대', '가', '반응', '할까', '요']","['이러고는', '형', '이', '확정', '된', '건', '아니니', '최종'..."
2,조국,조국장관 딸 표창장 위조 인정되었네요. 이제 학교당국의 후속조치 기대하겠습니다.,근데 아마 학교당국은 눈치보느라 아무것도 안하겠죠?,854,2020-12-23,이런건 고려대학교가 직접 나서서 먼저 제재 했어야 했는데 실망스럽습니다.\n판결을 ...,1550,3,근데 아마 학교 당국은 눈치 보느라 아무것도 안하겠죠,이런 건 고려대학교가 직접 나서서 먼저 제재 했어야 했는데 실망스럽습니다 판결을 떠...,"['근데', '아마', '학교', '당국', '은', '눈치', '보느라', '아무...","['이런', '건', '고려대학교', '가', '직접', '나서서', '먼저', '..."
3,조국,고려대학교는 죽었다.,1. 고려대 2010학년도 수시모집 요강의 수험생 공통 유의사항 : '서류 위조·변...,936,2020-12-30,권력의 개들이죠,415,1,1 고려대 2010학년도 수시 모집 요강의 수험생 공통 유의사항 서류 위조 변조 및...,권력의 개들이죠,"['1', '고려대', '2010', '학년', '도', '수시', '모집', '요...","['권력', '의', '개', '들이죠']"
4,조국,"고대 총장 및 교직원들, 뭐하고 있나요?","전 나라를 뒤흔든 2017년, 정유라 사태에선 학생들의 집단 요구로 이대 교직원 ...",446,2021-01-17,쪽팔려요 진심,150,1,전 나라를 뒤흔든 2017년 정유라 사태에선 학생들의 집단 요구로 이 대 교직원 및...,쪽팔려 요 진심,"['전', '나라', '를', '뒤', '흔든', '2017년', '정유', '라'...","['쪽팔려', '요', '진심']"


In [ ]:
del news["Unnamed: 0"]

In [ ]:
news.head()

,press,reporter,title,content,date,url,content_preprocess,content_tokenize
0,중앙일보,나운채,"정경심 유죄에 ""아직 판결문 읽나""…고대생들 분노 넘어 자조",“이젠 정진택 총장도 답변하실 때가 됐습니다” 서울 성북구 고려대학교 본관에 고려...,2022-01-28,https://www.joongang.co.kr/article/25044471,이 젠 정진택 총장도 답변 하실 때가 됐습니다 서울 성북구 고려대학교 본관에 고려대...,"['이', '젠', '정진', '택', '총장', '도', '답변', '하실', '..."
1,조선일보,김형원,조민 ‘고려대 입학취소’…정치권 설전으로 옮겨 붙었다,여야(與野)핵심의원들이 조국 전 법무부장관의 딸 조민씨의 고려대 입학취소 문제와 관...,2021-08-27,https://www.chosun.com/politics/assembly/2021/...,여야 핵심 의원들이 조국 전 법무부 장관의 딸 조민씨의 고려대 입학 취소 문제와 관...,"['여야', '핵심', '의원', '들', '이', '조국', '전', '법무부',..."
2,세계일보,정은나리,‘조민 입학 취소’ 논란 확산… 반대 청원 20만 돌파 VS ‘고려대→조려대’ 풍자,"부산대 민주동문회 “입학취소 성급, 즉각 철회하라” 고려대 커뮤니티선 “총장이 권력...",2021-08-26,http://www.segye.com/content/html/2021/08/26/2...,부산대 민주동문회 입학취소 성급 즉각 철회하라 고려대 커뮤니티 선 총장이 권력 눈치...,"['부산', '대', '민주', '동문회', '입학', '취소', '성', '급',..."
3,서울경제,박신원,"""조려대"" ""민족고대 아닌 Min Cho고대""…고대생들의 분노",고려대학교 로고(왼쪽)에 조국 전 법무부 장관 딸 조민씨로 추정되는 여성(오른쪽...,2021-08-26,http://www.sedaily.com/NewsView/22QC76NBP5,고려대학교 로고 왼쪽 에 조국 전 법무부 장관 딸 조민씨로 추정되는 여성 오른쪽 의...,"['고려대학교', '로고', '왼쪽', '에', '조국', '전', '법무부', '..."
4,머니투데이,박효주,"고려대 아닌 '조려대'…고대생의 분노, 조민 얼굴 합성해 조롱",고려대학교 로고(왼쪽)에 조국 전 법무부 장관 딸 조민씨 얼굴을 합성한 로고. /사...,2021-08-26,http://news.moneytoday.co.kr/view/mtview.php?n...,고려대학교 로고 왼쪽 에 조국 전 법무부 장관 딸 조민씨 얼굴을 합성한 로고 사진 ...,"['고려대학교', '로고', '왼쪽', '에', '조국', '전', '법무부', '..."


In [ ]:
comu.isnull().sum()

keyword               0
title                 2
content               5
likes                 0
date                  0
comment               8
comment_likes         0
comment_num           0
content_preprocess    1
comment_preprocess    8
content_tokenize      1
comment_tokenize      8
dtype: int64

In [ ]:
news.isnull().sum()

press                 0
reporter              6
title                 0
content               0
date                  0
url                   0
content_preprocess    0
content_tokenize      0
dtype: int64

# 대표 키워드 5개 추출하기 

##뉴스 키워드 추출

In [ ]:
news_c =news.copy() 

In [ ]:
test_c = test.copy()

In [ ]:
test_c["content_clean"][0]

' 이젠 정진택 총장도 답변하실 때가 됐습니다   서울 성북구 고려대학교 본관에 고려대 깃발이 바람에 펄럭이고 있다   뉴스1   지난 27일 고려대학교 학생 온라인 커뮤니티  고파스 에 올라온 글의 제목이다  조국 전 법무부 장관의 딸 조민씨의 입학취소 처분을 촉구하는 내용을 담고 있다  같은날 대법원은 조 전 장관의 부인 정경심 전 동양대 교수에 대해 징역 4년의 실형을 확정했다  해당 글은 28일 오전까지 1만 2000여 건의 조회 수를 기록하고 430건 이상의 추천을 받았다 조민  7대 허위스펙  1 2 3심 재판부 판단 그래픽 이미지  신재민 기자  고려대  원론적 입장 6개월째 진전은 없어     고려대는 지난해 8월부터 조씨 입학 취소 여부를 논의 중이지만  별다른 진전은 없는 상황이다  고려대는 이날도  입학취소처리심의위원회에서 규정과 절차에 따라 논의가 진행되고 있다 는 원론적인 답변을 내놨다  고려대 규정은 입학 사정을 위해 제출한 전형 자료에서 중대한 흠결이 발견될 경우 입학취소처리심의위에서 절차에 따라 처리한다고 돼 있다    조씨의 고려대 입시 업무방해 의혹은 공소시효가 지나 부산대 의학전문대학원 의전원 과 달리 정 전 교수 재판에서 직접적으로 다뤄지지 않았다  다만 조씨의 이른바  7대 허위 스펙 은 대법원에서 모두 유죄가 확정됐다  이 중 단국대 의과학연구소 인턴 활동 및 논문 등은 조씨의 고교 생활기록부에 반영돼 고려대 입학에 활용됐을 것으로 보인다  고려대 학생 온라인 커뮤니티  고파스  캡처    아직도 판결문 읽나  자조 섞인 반응 나와     일부 고려대 학생들은 온라인상에서 댓글 등을 달며  쪽팔린다 창피하다  는 등의 반응을 보였다   아직도 판결문을 읽고 있나  이 정도면  판결문을  외우겠다 는 등의 댓글도 있었다  앞서 정진택 고려대 총장이 지난해 6월 국민의힘 교육위원회 측과 면담하는 과정에서   정 전 교수  2심 판결 이후 관련 조치를 취하겠다 고 말했기 때문이다  두 달 뒤인 2심 판결 직후 고려대도  판결문

In [ ]:
news_c["content_preprocess"]

0      이 젠 정진택 총장도 답변 하실 때가 됐습니다 서울 성북구 고려대학교 본관에 고려대...
1      여야 핵심 의원들이 조국 전 법무부 장관의 딸 조민씨의 고려대 입학 취소 문제와 관...
2      부산대 민주동문회 입학취소 성급 즉각 철회하라 고려대 커뮤니티 선 총장이 권력 눈치...
3      고려대학교 로고 왼쪽 에 조국 전 법무부 장관 딸 조민씨로 추정되는 여성 오른쪽 의...
4      고려대학교 로고 왼쪽 에 조국 전 법무부 장관 딸 조민씨 얼굴을 합성한 로고 사진 ...
                             ...                        
246    최근 서울의 두 사립 명문대에서 서울 지방 캠퍼스 간 차별 논란과 학생들 간 갈등이...
247    최근 고려대 서울캠퍼스 총학생회 비상대책위원회 임원에 세종캠퍼스 소속 학생이 임명되...
248    정진택 고려대학교 총장이 8일 최근 불거진 서울 세종 캠퍼스 간 갈등과 관련해 절제...
249    고려대학교 서울캠퍼스 총학생회 임원으로 세종캠퍼스 소속 학생이 인준 받았다는 소식에...
250    최근 고려대 서울캠퍼스 총학생회 비상대책위원회 임원에 세종캠퍼스 소속 학생이 임명되...
Name: content_preprocess, Length: 251, dtype: object

In [ ]:
news_c["content_preprocess"][0]

'이 젠 정진택 총장도 답변 하실 때가 됐습니다 서울 성북구 고려대학교 본관에 고려대 깃발이 바람에 펄럭이고 있다 뉴스1 지난 27일 고려대학교 학생 온라인 커뮤니티 고파스 에 올라온 글의 제목이 다 조국 전 법무부 장관의 딸 조민씨의 입학 취소 처분을 촉구하는 내용을 담고 있다 같은 날 대법원은 조 전 장관의 부인 정경심 전 동양대 교수에 대해  징역 4년의 실형을 확정했다 해당 글은 28일 오전까지 1만 2000여 건의 조회 수를 기록하고 430건 이상의 추천을 받았다 조민 7대 허위스펙 1 2 3심 재판부 판단 그래픽 이미지 신재민 기자 고려대 원론적 입장 6개월째 진전은 없어 고려대는 지난해 8월부터 조씨 입학 취소 여부를 논의 중이지만 별다른 진전은 없는 상황이 다  고려대는 이 날도 입학 취소 처리심의위원회에서 규정과 절차에 따라 논의가 진행되고 있다 는 원론적인 답변을 내놨다 고려대 규정은 입학 사정을 위해 제출한 전형 자료에서 중대한 흠결이 발견될 경우 입학 취소 처리심의위에서 절차에 따라 처리한다고 돼 있다 조씨의 고려대 입 시 업무방해 의혹은 공소시효가 지나 부산대 의학 전문대학원 의전원 과 달리 정 전 교수  재판에서 직접적으로 다뤄지지 않았다 다만 조씨의 이른바 7대 허위 스펙 은 대법원에서 모두 유죄가 확정됐다 이 중 단국대 의과학연구소 인턴 활동 및 논문 등은 조씨의 고교 생활기록부에 반영돼 고려대 입학에 활용됐을 것으로 보인 다 고려대 학생 온라인 커뮤니티 고파스 캡처 아직도 판결문 읽나 자조 섞인 반응 나와 일부 고려대 학생들 은 온라인상에서 댓글 등을 달며 쪽 팔린 다 창피하다 는 등의 반응을 보였다 아직도 판결문을 읽고 있나 이 정도면 판결문을 외우겠다 는 등의 댓글도 있었다 앞서 정진택 고려대 총장이 지난해 6월 국민의 힘 교육위원회 측과 면담하는 과정에서 정 전 교수 2심 판결 이후 관련 조치를 취하겠다 고 말했기 때문이다 두 달 뒤인 2심 판결 직후  고려대도 판결문을 확보 검토한 후 본교의 학사 운영 규정에 따라 후속 조

### 뉴스 본문 중 명사만 추출

In [ ]:
okt = Okt()
tokenized_doc = okt.pos(news_c["content_preprocess"][0])
tokenized_doc

[('이', 'Noun'),
 ('젠', 'Noun'),
 ('정진', 'Noun'),
 ('택', 'Noun'),
 ('총장', 'Noun'),
 ('도', 'Josa'),
 ('답변', 'Noun'),
 ('하실', 'Verb'),
 ('때', 'Noun'),
 ('가', 'Josa'),
 ('됐습니다', 'Verb'),
 ('서울', 'Noun'),
 ('성북구', 'Noun'),
 ('고려대학교', 'Noun'),
 ('본관', 'Noun'),
 ('에', 'Josa'),
 ('고려대', 'Noun'),
 ('깃발', 'Noun'),
 ('이', 'Josa'),
 ('바람', 'Noun'),
 ('에', 'Josa'),
 ('펄럭', 'Noun'),
 ('이고', 'Josa'),
 ('있다', 'Adjective'),
 ('뉴스', 'Noun'),
 ('1', 'Number'),
 ('지난', 'Noun'),
 ('27일', 'Number'),
 ('고려대학교', 'Noun'),
 ('학생', 'Noun'),
 ('온라인', 'Noun'),
 ('커뮤니티', 'Noun'),
 ('고파스', 'Noun'),
 ('에', 'Josa'),
 ('올라온', 'Verb'),
 ('글', 'Noun'),
 ('의', 'Josa'),
 ('제목', 'Noun'),
 ('이', 'Josa'),
 ('다', 'Adverb'),
 ('조국', 'Noun'),
 ('전', 'Noun'),
 ('법무부', 'Noun'),
 ('장관', 'Noun'),
 ('의', 'Josa'),
 ('딸', 'Noun'),
 ('조', 'Modifier'),
 ('민씨', 'Noun'),
 ('의', 'Josa'),
 ('입학', 'Noun'),
 ('취소', 'Noun'),
 ('처분', 'Noun'),
 ('을', 'Josa'),
 ('촉구', 'Noun'),
 ('하는', 'Verb'),
 ('내용', 'Noun'),
 ('을', 'Josa'),
 ('담고', 'Verb'),
 ('있

In [ ]:
okt = Okt()
tokenized_doc = okt.pos(test_c["content_clean"][0])
tokenized_doc

[('이', 'Determiner'),
 ('젠', 'Noun'),
 ('정진', 'Noun'),
 ('택', 'Noun'),
 ('총장', 'Noun'),
 ('도', 'Josa'),
 ('답변', 'Noun'),
 ('하실', 'Verb'),
 ('때', 'Noun'),
 ('가', 'Josa'),
 ('됐습니다', 'Verb'),
 ('서울', 'Noun'),
 ('성북구', 'Noun'),
 ('고려대학교', 'Noun'),
 ('본관', 'Noun'),
 ('에', 'Josa'),
 ('고려대', 'Noun'),
 ('깃발', 'Noun'),
 ('이', 'Josa'),
 ('바람', 'Noun'),
 ('에', 'Josa'),
 ('펄럭', 'Noun'),
 ('이고', 'Josa'),
 ('있다', 'Adjective'),
 ('뉴스', 'Noun'),
 ('1', 'Number'),
 ('지난', 'Noun'),
 ('27일', 'Number'),
 ('고려대학교', 'Noun'),
 ('학생', 'Noun'),
 ('온라인', 'Noun'),
 ('커뮤니티', 'Noun'),
 ('고파스', 'Noun'),
 ('에', 'Josa'),
 ('올라온', 'Verb'),
 ('글', 'Noun'),
 ('의', 'Josa'),
 ('제목', 'Noun'),
 ('이다', 'Josa'),
 ('조국', 'Noun'),
 ('전', 'Noun'),
 ('법무부', 'Noun'),
 ('장관', 'Noun'),
 ('의', 'Josa'),
 ('딸', 'Noun'),
 ('조', 'Modifier'),
 ('민씨', 'Noun'),
 ('의', 'Josa'),
 ('입학', 'Noun'),
 ('취소', 'Noun'),
 ('처분', 'Noun'),
 ('을', 'Josa'),
 ('촉구', 'Noun'),
 ('하는', 'Verb'),
 ('내용', 'Noun'),
 ('을', 'Josa'),
 ('담고', 'Verb'),
 ('있다', 'Adject

In [ ]:
def picknoun(text):
    okt = Okt()
    tokenized_doc = okt.pos(text) #품사 구별 
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun']) 
    #명사 단어만 뽑아서 문자열화
    return tokenized_nouns

### CountVectorizer 사용해 단어 추출

In [ ]:
def wordextraction(noun : list) -> list:
    n_gram_range = (2, 3) #biagram(2개 1단어), trigram(3개 1단어) 추출

    count = CountVectorizer(ngram_range=n_gram_range).fit(noun)
    candidates = count.get_feature_names_out()

    return candidates

###SBERT를 통한 키워드 수치화

In [ ]:
doc = """
드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
"향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
"""

In [ ]:
[doc]

['\n드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.\n서울시는 \'미세먼지 계절관리제\' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.\n드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.\n영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.\n드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.\n지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.\n이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.\n소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,\n유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며\n"향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.\n']

In [ ]:
news_c["content_preprocess"][50]

'고려대 재학생 및 졸업생들이 지난 8월 23일 교내에서 조국 당시 법무부 장관 후보자 딸의 입학 과정에 대한 진상규명을 촉구하며 행진하고 있다 이병주 기자 고려대 학생들이 조국 전 법무부 전 장관 딸의 입학 취소 문제를 두고 계속 말을 바꾸고 있는 학교 측에 반발해 조국 딸 입학 취소 집회를 추진한 다 정진택 고려대 총장이 지난 15일 사실 관계가  확실히 밝혀질 때까지 기다린다 는 취지의 입장문을 발표한 후 비판은 더 거세지고 있다 한 시민단체는 정 총장이 조씨의 입학 취소를 거부하고 있다며 검찰에 고발하기로 했다 집회 집행부를 이끄는 고려대 재학생 a씨는 17일 국민일보와 통화에서 학교가 조씨 의혹과 관련해 사실과 맞지 않는 주장을 하며 제대로 된 조치를 취하지 않고 있다 며 이달 22일에  교내에서 조씨 입학 취소를 촉구하는 집회를 개최할 계획 이라고 말했다 고려대 온라인 커뮤니티 등에서 자원봉사자를 모은 a씨는 18일부터 집회 관련 대자보를 붙이고 본격적으로 시 위를 추진할 예정이다 학생들은 검찰 수사 결과 조씨의 허위 위조 경력이 고려대 입학 과정에 사용된 것이 확인됐음에도 학교가 미적거리는 데 불만을 갖고 있다 정경심 교수에 관한  검찰 공소장에 따르면 딸 조씨가 1저자로 등재된 단국대 병리학 논문과 3저자로 오른 공주대 논문 초록 서울대 공익인권법센터 인턴 경력 등은 생활기록부에 기재돼 대학 입시에 활용됐다 검찰은 이를 조 전 장관의 지위와 인맥으로 만든 허위 스펙 으로 보고 있다 정 총장은 하지만 본교 입학에 활용됐다는 입 시 자료가 실제로 제출됐는지 알 수 없다 며  학교 측이 전형 자료를 확인할 수 없는 상황에서 취할 수 있는 조치가 없다 고 주장했다 정 총장은 검찰의 공소 사실에 고려대 입학 관련 내용은 포함되지 않았다 는 이야기도 덧붙였다 검찰은 공소시효를 감안해 10년 전 고려대 입 시 관련 혐의는 기소하지 않았다 고려대는 조씨 입학 취소에 관해 입장을 바꾼 적이 없다고 했지만 실제로는 조금씩 말이 달라졌다   지난 8월 조씨의 대 입

In [ ]:
[picknoun(news_c["content_preprocess"][50])]

['고려대 재학생 및 졸업생 지난 교내 조국 당시 법무부 장관 후보자 딸 입학 과정 대한 진상 규명 촉구 행진 이병주 기자 고려대 학생 조국 전 법무부 전 장관 딸 입학 취소 문제 계속 말 학교 측 반발 조국 딸 입학 취소 집회 추진 정진 택 고려대 총장 지난 사실 관계 때 취지 입장 문 발표 후 비판 더 시민단체 정 총장 조씨 입학 취소 거부 검찰 고발 집회 집행 고려대 재학생 씨 국민일보 통화 학교 조씨 의혹 관련 사실 주장 제대로 조치 취하 며 달 교내 조씨 입학 취소 촉구 집회 개최 계획 말 고려대 온라인 커뮤니티 등 자원봉사자 씨 집회 관련 대자보 붙이 본격 시 위 추진 예정 학생 검찰 수사 결과 조씨 허위 위조 경력 고려대 입학 과정 사용 것 확인 학교 미적 거리 데 불만 정경 심 교수 관 검찰 공소장 딸 조씨 저자 등재 단국대 병리학 논문 저자 오른 공주대 논문 초록 서울대 공익 권법 센터 인턴 경력 등 생활기록부 기재 대학 입시 활용 검찰 조 전 장관 지위 인맥 허위 스펙 보고 정 총장 본교 입학 활용 입 시 자료 실제 제출 알 수 며 학교 측 전형 자료 확인 수 상황 취할 수 조치 고 주장 정 총장 검찰 공소 사실 고려대 입학 관련 내용 포함 이야기 검찰 시효 감안 전 고려대 입 시 관련 혐의 기소 고려대 조씨 입학 취소 관해 입장 적 실제 말 지난 조씨 입 문제 처음 때 고려대 논문 문제 점 절차 입학 취소 등 진행 고 병리학 회 논문 취소 이후 검찰 수사 며 공 검찰 바 이 제 수사 결과 입학 취소 대한 입장 불분명 상황 시민단체 법치 주의 바로 행동 연대 정 총장 업무방해 명예훼손 혐의 검찰 고발 단체 검찰 수사 통해 조씨 입 시 비리 고려대 궤변 입학 취소 거부 며 불공정 분 노 국민 학생 우롱 것 주장']

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

###상위 5개 키워드 출력(기본 Keybert)

In [ ]:
keyword_corpse=[]
for content in news_c["content_preprocess"]:
    candidates = wordextraction([picknoun(content)])
    doc_embedding = model.encode(list(content)) #list화 시켜야 함
    candidate_embeddings = model.encode(candidates)
    top_n = 5
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates [index] for index in distances.argsort()[0][-top_n:]]
    print(keywords)
    keyword_corpse.append(keywords)

news_c["keyword_base"] = keyword_corpse

['위해', '승호', '대해', '상황', '중이']
['리지', '제출', '몰이', '안고', '여야']
['거세', '예비', '대의', '고파스', '전형']
['합격', '등장', '상황', '파스', '취할']
['처음', '등장', '상황', '예비', '파스']
['예비', '파스', '취할', '사정', '해당']
['예비', '대의', '라며', '제기', '해당']
['대의', '라며', '제기', '정씨', '다만']
['안내', '이후', '하단', '대해', '가운데']
['합격', '상황', '예비', '파스', '취할']
['등장', '상황', '파스', '대의', '제기']
['등장', '상황', '예비', '취할', '리지']


KeyboardInterrupt: ignored

5개의 키워드가 의미가 거의 비슷한 이유: 문서를 가장 잘 나타내고 있기 때문에 <br>
좀 더 다양한 의미의 키워드들이 출력된다면 이들을 그룹으로 본다는 관점에서는 문서를 잘 나타낼 가능성이 적을 수 있다 <br>
키워드 선정의 정확성과 키워드들의 다양성 사이의 균형이 필요!!

### Max Sum Similarity

데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. <br>여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.

In [ ]:
df = pd.DataFrame({""})

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
keyword2_corpse = []
for content in news_c["content_preprocess"]:
    candidates = wordextraction([picknoun(content)])
    doc_embedding = model.encode(list(content)) #list화 시켜야 함
    candidate_embeddings = model.encode(candidates)
    keyword_max = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)
    print(keyword_max)
    keyword2_corpse.append(keyword_max)

news_c["keyword_mss"] = keyword2_corpse

['거나 조씨', '취소 여부', '위해 제출', '총장 답변', '확정 해당']
['센터 인턴', '조씨 입학', '앞서 부산', '여야 핵심 의원', '조씨 얼굴']
['보나 조국', '표창장 합격', '즉각 철회 모든', '취소 여부 관련', '판단 대학']
['취소 결정', '오른쪽 얼굴', '모두 허위', '상황 오후', '등장 조씨']
['계기 다시 등장', '모두 허위', '등장 조씨', '구성 향후 추가', '취소 예비']
['취소 예비', '조씨 허위', '하나 자리', '스펙 모두', '수기 합격']
['해당 게시', '하루 이상', '대신 조씨', '취소 요구', '이상 동의']
['행태 상황', '납득 대부분', '사실 확인', '조씨 허위', '말로 부정']
['조씨 입학 취소', '조씨 입학', '해당 로고', '원래 호랑이', '가운데 조씨']
['민씨 오른쪽', '상황 안내 입장', '조씨 얼굴', '모두 허위', '하나 자리']
['총장 앞서', '학생 합격', '취소 결정', '취하 조치', '계기 다시']
['진행 상황', '모두 허위', '풍자 조씨', '구성 향후 추가', '취소 예비']
['상이 합격', '무리 합격', '응시 황도', '부산 책임', '추정 지난']
['오후 조씨 입학', '다음 적발', '저자 등재', '현재 조씨', '취소 주제']
['취소 책임', '상이 합격', '무리 합격', '입시 최대로', '부산 책임']
['통과 전원', '합격 수기 제목', '조씨 허위', '취소 여부', '조씨 학년']
['조씨 부산', '상황 입학 취소', '법무 법인 인턴', '합격 한국전력', '센터 인턴']
['확인 개월 동안', '라며 의원', '불법 안하무인', '취소 검토', '입시 부정']
['처리 해당', '입학 전형', '의사 국가', '취소 게시 다른', '취소 요구']
['이번 사건', '사이 입장', '조씨 허위', '취소 결정', '그것 모자라']
['총장 맙시 총장', '확인 동양대 총장', 

In [ ]:
news_c.head()

,press,reporter,title,content,date,url,content_preprocess,content_tokenize,keyword_base,keyword_mss
0,중앙일보,나운채,"정경심 유죄에 ""아직 판결문 읽나""…고대생들 분노 넘어 자조",“이젠 정진택 총장도 답변하실 때가 됐습니다” 서울 성북구 고려대학교 본관에 고려...,2022-01-28,https://www.joongang.co.kr/article/25044471,이 젠 정진택 총장도 답변 하실 때가 됐습니다 서울 성북구 고려대학교 본관에 고려대...,"['이', '젠', '정진', '택', '총장', '도', '답변', '하실', '...","[다만 조씨, 입학 사정, 확정 부산, 주체 대한, 해당 오전]","[거나 조씨, 취소 여부, 위해 제출, 총장 답변, 확정 해당]"
1,조선일보,김형원,조민 ‘고려대 입학취소’…정치권 설전으로 옮겨 붙었다,여야(與野)핵심의원들이 조국 전 법무부장관의 딸 조민씨의 고려대 입학취소 문제와 관...,2021-08-27,https://www.chosun.com/politics/assembly/2021/...,여야 핵심 의원들이 조국 전 법무부 장관의 딸 조민씨의 고려대 입학 취소 문제와 관...,"['여야', '핵심', '의원', '들', '이', '조국', '전', '법무부',...","[인턴 확인 동양대, 확인 공주대, 센터 인턴 확인, 호텔 인턴 확인, 인턴 확인 ...","[센터 인턴, 조씨 입학, 앞서 부산, 여야 핵심 의원, 조씨 얼굴]"
2,세계일보,정은나리,‘조민 입학 취소’ 논란 확산… 반대 청원 20만 돌파 VS ‘고려대→조려대’ 풍자,"부산대 민주동문회 “입학취소 성급, 즉각 철회하라” 고려대 커뮤니티선 “총장이 권력...",2021-08-26,http://www.segye.com/content/html/2021/08/26/2...,부산대 민주동문회 입학취소 성급 즉각 철회하라 고려대 커뮤니티 선 총장이 권력 눈치...,"['부산', '대', '민주', '동문회', '입학', '취소', '성', '급',...","[여부 관련 전날, 등장 고파스, 대해 공식, 것임 반증, 확정 처분]","[보나 조국, 표창장 합격, 즉각 철회 모든, 취소 여부 관련, 판단 대학]"
3,서울경제,박신원,"""조려대"" ""민족고대 아닌 Min Cho고대""…고대생들의 분노",고려대학교 로고(왼쪽)에 조국 전 법무부 장관 딸 조민씨로 추정되는 여성(오른쪽...,2021-08-26,http://www.sedaily.com/NewsView/22QC76NBP5,고려대학교 로고 왼쪽 에 조국 전 법무부 장관 딸 조민씨로 추정되는 여성 오른쪽 의...,"['고려대학교', '로고', '왼쪽', '에', '조국', '전', '법무부', '...","[총장 대한, 확정 관련, 취하 상황, 취소 처리, 합격 취소]","[취소 결정, 오른쪽 얼굴, 모두 허위, 상황 오후, 등장 조씨]"
4,머니투데이,박효주,"고려대 아닌 '조려대'…고대생의 분노, 조민 얼굴 합성해 조롱",고려대학교 로고(왼쪽)에 조국 전 법무부 장관 딸 조민씨 얼굴을 합성한 로고. /사...,2021-08-26,http://news.moneytoday.co.kr/view/mtview.php?n...,고려대학교 로고 왼쪽 에 조국 전 법무부 장관 딸 조민씨 얼굴을 합성한 로고 사진 ...,"['고려대학교', '로고', '왼쪽', '에', '조국', '전', '법무부', '...","[총장 대한, 달리 미온, 확정 관련, 취소 처리, 합격 취소]","[계기 다시 등장, 모두 허위, 등장 조씨, 구성 향후 추가, 취소 예비]"


In [ ]:
del news_c["content_tokenize"]

In [ ]:
news_c.head()

,press,reporter,title,content,date,url,content_preprocess,keyword_base,keyword_mss
0,중앙일보,나운채,"정경심 유죄에 ""아직 판결문 읽나""…고대생들 분노 넘어 자조",“이젠 정진택 총장도 답변하실 때가 됐습니다” 서울 성북구 고려대학교 본관에 고려...,2022-01-28,https://www.joongang.co.kr/article/25044471,이 젠 정진택 총장도 답변 하실 때가 됐습니다 서울 성북구 고려대학교 본관에 고려대...,"[다만 조씨, 입학 사정, 확정 부산, 주체 대한, 해당 오전]","[거나 조씨, 취소 여부, 위해 제출, 총장 답변, 확정 해당]"
1,조선일보,김형원,조민 ‘고려대 입학취소’…정치권 설전으로 옮겨 붙었다,여야(與野)핵심의원들이 조국 전 법무부장관의 딸 조민씨의 고려대 입학취소 문제와 관...,2021-08-27,https://www.chosun.com/politics/assembly/2021/...,여야 핵심 의원들이 조국 전 법무부 장관의 딸 조민씨의 고려대 입학 취소 문제와 관...,"[인턴 확인 동양대, 확인 공주대, 센터 인턴 확인, 호텔 인턴 확인, 인턴 확인 ...","[센터 인턴, 조씨 입학, 앞서 부산, 여야 핵심 의원, 조씨 얼굴]"
2,세계일보,정은나리,‘조민 입학 취소’ 논란 확산… 반대 청원 20만 돌파 VS ‘고려대→조려대’ 풍자,"부산대 민주동문회 “입학취소 성급, 즉각 철회하라” 고려대 커뮤니티선 “총장이 권력...",2021-08-26,http://www.segye.com/content/html/2021/08/26/2...,부산대 민주동문회 입학취소 성급 즉각 철회하라 고려대 커뮤니티 선 총장이 권력 눈치...,"[여부 관련 전날, 등장 고파스, 대해 공식, 것임 반증, 확정 처분]","[보나 조국, 표창장 합격, 즉각 철회 모든, 취소 여부 관련, 판단 대학]"
3,서울경제,박신원,"""조려대"" ""민족고대 아닌 Min Cho고대""…고대생들의 분노",고려대학교 로고(왼쪽)에 조국 전 법무부 장관 딸 조민씨로 추정되는 여성(오른쪽...,2021-08-26,http://www.sedaily.com/NewsView/22QC76NBP5,고려대학교 로고 왼쪽 에 조국 전 법무부 장관 딸 조민씨로 추정되는 여성 오른쪽 의...,"[총장 대한, 확정 관련, 취하 상황, 취소 처리, 합격 취소]","[취소 결정, 오른쪽 얼굴, 모두 허위, 상황 오후, 등장 조씨]"
4,머니투데이,박효주,"고려대 아닌 '조려대'…고대생의 분노, 조민 얼굴 합성해 조롱",고려대학교 로고(왼쪽)에 조국 전 법무부 장관 딸 조민씨 얼굴을 합성한 로고. /사...,2021-08-26,http://news.moneytoday.co.kr/view/mtview.php?n...,고려대학교 로고 왼쪽 에 조국 전 법무부 장관 딸 조민씨 얼굴을 합성한 로고 사진 ...,"[총장 대한, 달리 미온, 확정 관련, 취소 처리, 합격 취소]","[계기 다시 등장, 모두 허위, 등장 조씨, 구성 향후 추가, 취소 예비]"


In [ ]:
news_c.to_excel("뉴스 키워드 추출.xlsx")

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

###Maximal Marginal Relevance

텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다<Br>
먼저 문서와 가장 유사한 키워드/키프레이즈를 선택합니다. <br>그런 다음 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택합니다.

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings,candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
for content in news_c["content_preprocess"]:
    candidates = wordextraction([picknoun(content)])
    print(len(candidates))
    doc_embedding = model.encode(list(content)) #list화 시켜야 함
    candidate_embeddings = model.encode(candidates)
    # print(mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7))

315
232
263


KeyboardInterrupt: ignored

## 커뮤니티 키워드 추출

In [ ]:
comu_c = comu.copy()

In [ ]:
comu_c.isnull().sum()
#content에 1개, comment에 8개

keyword               0
title                 2
content               5
likes                 0
date                  0
comment               8
comment_likes         0
comment_num           0
content_preprocess    1
comment_preprocess    8
content_tokenize      1
comment_tokenize      8
dtype: int64

In [ ]:
comu_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   keyword             109 non-null    object
 1   title               107 non-null    object
 2   content             104 non-null    object
 3   likes               109 non-null    int64 
 4   date                109 non-null    object
 5   comment             101 non-null    object
 6   comment_likes       109 non-null    int64 
 7   comment_num         109 non-null    int64 
 8   content_preprocess  108 non-null    object
 9   comment_preprocess  101 non-null    object
 10  content_tokenize    108 non-null    object
 11  comment_tokenize    101 non-null    object
dtypes: int64(3), object(9)
memory usage: 10.3+ KB


In [ ]:
comu_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   keyword             109 non-null    object
 1   title               107 non-null    object
 2   content             104 non-null    object
 3   likes               109 non-null    int64 
 4   date                109 non-null    object
 5   comment             101 non-null    object
 6   comment_likes       109 non-null    int64 
 7   comment_num         109 non-null    int64 
 8   content_preprocess  108 non-null    object
 9   comment_preprocess  101 non-null    object
 10  content_tokenize    108 non-null    object
 11  comment_tokenize    101 non-null    object
dtypes: int64(3), object(9)
memory usage: 10.3+ KB


### 기본 keybert방식

In [ ]:
comu_c["content_preprocess"][5]

nan

In [ ]:
def picknoun(text):
    okt = Okt()
    tokenized_doc = okt.pos(text) #품사 구별 
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun']) 
    #명사 단어만 뽑아서 문자열화
    return tokenized_nouns

In [ ]:
def wordextraction(noun : list) -> list:
    n_gram_range = (1,3) #biagram(2개 1단어), trigram(3개 1단어) 추출

    count = CountVectorizer(ngram_range=n_gram_range).fit(noun)
    candidates = count.get_feature_names_out()

    return candidates

In [ ]:
keyword_corpse=[]
for content in comu_c["content_preprocess"]:
    try:
        candidates = wordextraction([picknoun(content)])
        doc_embedding = model.encode(list(content)) #list화 시켜야 함
        candidate_embeddings = model.encode(candidates)

        top_n = 5
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [candidates [index] for index in distances.argsort()[0][-top_n:]]
        print(keywords)

        keyword_corpse.append(keywords)

    except: #np.nan일때

        keyword_corpse.append(content)


comu_c["content_keyword_base"] = keyword_corpse

['진짜 얘기', '진짜', '물어 진짜', '얘기', '물어']
['반응', '이번 고려대 반응', '고려대', '이번 고려대', '이번']
['눈치', '아마 학교', '아마 학교 당국', '아마', '당국']
['구체', '불구', '취소 오늘', '학년', '경우']
['질도', '그동안', '대해', '원금', '포함']
['본인 인간', '지금 우리', '총장 본인 희생', '총장 고민', '가족 지금']
['취소 논의', '총장 아래', '대도', '다른', '저희']
['이제 고려대 차례', '고려대', '차례', '이제 고려대', '이제']
['열쇠 민트 초코', '민트 초코', '지나', '초코 지나', '초코']
['라면 정권', '상식', '라면', '부장', '무마']
['민주당 인사', '민주당', '인사']
['월급 재계약', '복귀', '로스쿨 복귀', '재계약', '로스쿨']
['장소 도착 정말', '주신', '분명', '도착 정말', '사실']
['자신', '분과', '전형', '참으로', '미명 아래']
['고려대 응답', '고려대', '응답']
['사과', '국민', '진심', '가지', '여러']
['사명 거임', '원래', '완성', '레이저 원래', '거임']
['라며', '최대', '저장', '세운', '자가']
['재주']
['수호', '이건', '상황', '이쯤 검찰', '이쯤']
['옹호', '통제 완전', '집합', '보수', '완전']
['근거', '진술', '총장', '이수', '주도']
['무소', '하루 이틀', '명의', '학년', '피셜']
['수사', '목소리', '관련 의혹 대한', '대한', '의혹 대한']
['질문 조국 본인', '조국 본인', '본인', '본인 인정', '인정']
['교우 추천', '공채', '진로', '추천', '응시']
['하나요', '요즘', '박성민 요즘', '박성민', '박성민 요즘 하나요']
['신지 예인', '가요', '재앙 신지', '신지', '예인 가요']
[

In [ ]:
keyword_corpse=[]
for content in comu_c["comment_preprocess"]:
    try:
        candidates = wordextraction([picknoun(content)])
        doc_embedding = model.encode(list(content)) #list화 시켜야 함
        candidate_embeddings = model.encode(candidates)

        top_n = 3
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [candidates [index] for index in distances.argsort()[0][-top_n:]]
        print(keywords)

        keyword_corpse.append(keywords)

    except: #np.nan일때
        keyword_corpse.append(content)


comu_c["comment_keyword_base"] = keyword_corpse

['집회 누가', '추미애', '수호']
['확정', '확정 최종', '햇으']
['직접', '자체', '얘기']
['권력']
['진심']
['비리 말씀', '비리', '말씀']
['얼마나', '보고 결정', '보고']
['학생 침묵', '학생', '침묵']
['아무', '아무 저런', '저런']
['라면', '미명', '지랄']
['대체 나라', '나라', '대체']
['진짜', '구속', '깜빵']
['양심', '정말 양심', '정말']
['수고', '인원', '주최']
['다나']
['본인', '민아', '김어준']
['사죄 행여', '행여', '진심']
['파면 우리', '우리', '구속']
['수준 지금', '지금', '알도 비상']
['서초동', '국민 데리', '데리']
['아가리', '아가리 스테이플러', '민아']
['정도', '상황', '구도']
['깨문', '센터', '센터 인턴']
['이면', '차라리', '카페']
['그게', '이크 이쯤', '이쯤']
['여기 위증', '거지', '마냥']
['진지', '오빠', '거지']
['요새']
['그냥 윤석열', '대통령 그냥', '그냥']
['대표', '대표 저기', '저기']
['주작', '좌파 건가', '건가']
['하랴', '어익후', '차라리']
['가장', '배틀 아마', '이자']
['교수 대학원생', '논문', '교수']
['본교 문과', '세종', '본교']
['선거']
['비서', '그냥 사퇴', '세금']
['쿼터 쟨데', '너희', '쟨데']
['실제 진짜', '진짜', '텄네']
['일말', '애초', '호소']
['깨달 지경', '깨달', '지경']
['임명', '발탁', '위원']
['거의 진짜', '잘해야', '진짜']
['자신', '나이 알다', '알다']
['생각 신앙', '생각', '영역']
['보고 구나', '구나', '보고']
['동안', '포인트', '의식 포인트']
['강남대', '무언가 그것', '그것']
['방법 자기', '원래', '굳이']
['갈

###Max SUm Similarity

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
keyword2_corpse = []
for content in comu_c["content_preprocess"]:
    try:
        candidates = wordextraction([picknoun(content)])
        doc_embedding = model.encode(list(content)) #list화 시켜야 함
        candidate_embeddings = model.encode(candidates)
        keyword_max = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)
        print(keyword_max)
        keyword2_corpse.append(keyword_max)
    except:
        keyword2_corpse.append(content)

comu_c["content_keyword_mss"] = keyword2_corpse

['물어 진짜 얘기', '진짜 얘기', '진짜', '얘기', '물어']
['고려대 반응', '반응', '이번 고려대 반응', '고려대', '이번']
['학교 당국 눈치', '학교', '당국 눈치', '아마', '당국']
['모집', '이제', '내용 확인', '대리 시험', '취소 오늘']
['원금 달라', '절대 밀리', '얼굴', '모두', '그녀']
['안암', '지금', '이야기 본인 인간', '총장 고민 이해', '배알']
['주최', '입학', '그게', '경관', '아래']
['고려대 차례', '고려대', '차례', '이제 고려대', '이제']
['황금 열쇠', '지나 졸업식', '졸업식', '열쇠 민트', '민트']
['추미애 장관', '조국', '누가', '무혐의 상식', '윤석']
['월급 재계약 서울대', '서울대', '월급', '월급 재계약', '복귀']
['인증', '카페', '정말 정도', '때문', '모든']
['사퇴', '고초 유명', '달리 불의 전형', '유명', '잘못']
['여러 가지 요인', '진심 사과 요구', '국민 진심', '사퇴', '누구']
['조국 지지자', '조국 지지자 검찰청', '아래 문장 레이저', '문장 완성', '사명']
['리가', '주요', '정도 당신', '사실', '가장']
['조국 수호', '논리', '광화문', '집단', '반대']
['언론 통제', '진보 보수 세력', '독재정', '조국 옹호 진보', '완전']
['영어', '프로그램', '질문', '동양대 총장', '캠프']
['부디 당신', '하루', '그것', '소고기 아무', '미국 소고기 아무']
['개혁', '검찰 행사 무엇', '존중', '목소리 성찰', '대통령']
['정부 질문', '정부', '질문', '조국', '인정']
['시국 박성민 교우', '박성민 교우', '시국', '추천 진로', '응시']
['요즘 하나요', '하나요', '요즘', '박성민 요즘', '박성민']
['진짜 나라', '윤석열', '차이점 뭔가 하필

In [ ]:
keyword2_corpse = []
for content in comu_c["comment_preprocess"]:
    try:
        candidates = wordextraction([picknoun(content)])
        doc_embedding = model.encode(list(content)) #list화 시켜야 함
        candidate_embeddings = model.encode(candidates)
        keyword_max = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=3, nr_candidates=30)
        print(keyword_max)
        keyword2_corpse.append(keyword_max)
    except:
        keyword2_corpse.append(content)

comu_c["comment_keyword_mss"] = keyword2_corpse

['확산 집회', '논리', '선택']
['고대 타이틀', '졸업', '자퇴']
['판결', '세력', '고려대학교']
['닥치고 비리', '사과', '총장']
['판결 대법원', '학교', '얼마나']
['학생 침묵', '학생', '침묵']
['정의 진리', '인간 정권', '아무']
['검찰', '파장 수사', '사실 관계']
['위조 관행', '나라 소방관', '대체']
['박근혜', '박살', '진짜 진성']
['양심', '정말 양심', '정말']
['인원 고생', '여러분', '집회']
['정말 상관', '상관', '민아']
['전략', '대통령 지지율', '후회']
['정권 심판', '남불 좌파', '우리']
['문제 범죄자', '법무부 장관', '알도']
['검찰 압박', '독립', '무엇 때문']
['댓글', '아가리 스테이플러', '민아']
['누리 조기', '여당', '그름']
['서울대', '인권 센터', '조민아']
['조작 카페', '누리', '이해 자기']
['진짜 검찰', '총장 임명', '사람']
['유린', '찬스', '금수']
['스터디', '계속 대선', '이낙연 위치']
['공산주의', '강성 대국', '차이']
['코로나 스펙', '사태', '인사']
['주작 좌파', '페미 주작', '건가']
['초월', '비서', '무슨']
['정치', '라이벌 이준석', '이자']
['대학원생', '논문', '교수']
['문과', '계속', '세종']
['청년 강연', '어디', '시민단체 세금']
['청년 정책', '청와대', '남녀']
['남녀 문제', '설계', '실제']
['페미니즘 성범죄', '애초 민주당', '신장']
['깨달 지경', '깨달', '지경']
['청년 대변인', '위원 발탁', '최고']
['정책', '진짜 투표', '인사']
['자신 나라', '나라 나이', '알다']
['신앙', '생각', '영역']
['청년 정책', '정책', '구나']
['투쟁', '학벌 조롱', '동안']
['하버드 국가', '학벌'

In [ ]:
comu_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   keyword               109 non-null    object
 1   title                 107 non-null    object
 2   content               104 non-null    object
 3   likes                 109 non-null    int64 
 4   date                  109 non-null    object
 5   comment               101 non-null    object
 6   comment_likes         109 non-null    int64 
 7   comment_num           109 non-null    int64 
 8   content_preprocess    108 non-null    object
 9   comment_preprocess    101 non-null    object
 10  content_keyword_mss   108 non-null    object
 11  comment_keyword_base  101 non-null    object
 12  comment_keyword_mss   101 non-null    object
 13  content_keyword_base  104 non-null    object
dtypes: int64(3), object(11)
memory usage: 12.0+ KB


In [ ]:
comu_c.iloc[1,1]

'[속보]법원 "조국 딸, 단국대 1저자 논문 기여 안했다"'

In [ ]:
count=0
for i,content in enumerate(comu_c["content_keyword_base"]):
    if type(content) == list:
        count+=1
    else:
        comu_c.iloc[i,13] = np.nan
count

104

In [ ]:
count=0
for i,content in enumerate(comu_c["content_keyword_mss"]):
    if type(content) == list:
        count+=1
    else:
        comu_c.iloc[i,10] = np.nan
count

98

In [ ]:
count=0
for i,content in enumerate(comu_c["comment_keyword_base"]):
    if type(content) == list:
        count+=1
    else:
        comu_c.iloc[i,11] = np.nan
count

98

In [ ]:
count=0
for i,content in enumerate(comu_c["comment_keyword_mss"]):
    if type(content) == list:
        count+=1
    else:
        comu_c.iloc[i,12] = np.nan
count

92

In [ ]:
comu_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   keyword               109 non-null    object
 1   title                 107 non-null    object
 2   content               104 non-null    object
 3   likes                 109 non-null    int64 
 4   date                  109 non-null    object
 5   comment               101 non-null    object
 6   comment_likes         109 non-null    int64 
 7   comment_num           109 non-null    int64 
 8   content_preprocess    108 non-null    object
 9   comment_preprocess    101 non-null    object
 10  content_keyword_mss   98 non-null     object
 11  comment_keyword_base  98 non-null     object
 12  comment_keyword_mss   92 non-null     object
 13  content_keyword_base  104 non-null    object
dtypes: int64(3), object(11)
memory usage: 12.0+ KB


In [ ]:
comu_c.columns

Index(['keyword', 'title', 'content', 'likes', 'date', 'comment',
       'comment_likes', 'comment_num', 'content_preprocess',
       'comment_preprocess', 'content_keyword_mss', 'comment_keyword_base',
       'comment_keyword_mss', 'content_keyword_base'],
      dtype='object')

In [ ]:
comu_c = comu_c[['keyword', 'title', 'content', 'likes', 'date', 'comment',
       'comment_likes', 'comment_num', 'content_preprocess','content_keyword_base','content_keyword_mss',
       'comment_preprocess','comment_keyword_base','comment_keyword_mss']]

In [ ]:
comu_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   keyword               109 non-null    object
 1   title                 107 non-null    object
 2   content               104 non-null    object
 3   likes                 109 non-null    int64 
 4   date                  109 non-null    object
 5   comment               101 non-null    object
 6   comment_likes         109 non-null    int64 
 7   comment_num           109 non-null    int64 
 8   content_preprocess    108 non-null    object
 9   content_keyword_base  104 non-null    object
 10  content_keyword_mss   98 non-null     object
 11  comment_preprocess    101 non-null    object
 12  comment_keyword_base  98 non-null     object
 13  comment_keyword_mss   92 non-null     object
dtypes: int64(3), object(11)
memory usage: 12.0+ KB


In [ ]:
# del comu_c["content_tokenize"]
# del comu_c["comment_tokenize"]
del comu_c["content_keyword_base"]

In [ ]:
comu_c.to_excel("커뮤니티 키워드 추출(수정).xlsx")